## SPRi AI Brief_10월호_산업동향_F 을 분석해보자!

### 1. 문서 로드 (Load Documents)

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.document_loaders import PyMuPDFLoader

# PyMuPDFLoader 객체 정의
loader = PyMuPDFLoader("data/SPRi AI Brief_10월호_산업동향_F.pdf")

# 문서 로드
docs = loader.load()

print(f"문서의 페이지수 : {len(docs)}")

문서의 페이지수 : 25


## 2. 문서 분할(Split Documents)

In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

split_documents = text_splitter.split_documents(docs)

print(f"분할된 청크의 수 : {len(split_documents)}")

분할된 청크의 수 : 47


## 3. 임베딩(Embedding 생성)

In [4]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

## 4. DB 생성(벡터스토어 생성) 및 저장
- FAISS(Facebook AI Similarity Search)
    - 페이스북에서 개발한 유사도 검색 및 클러스터링 라이브러리
    - 벡터 데이터셋에서 빠른 유사도 검색

In [24]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [6]:
for doc in vectorstore.similarity_search("옥스퍼드"):
    print(doc.page_content)

3. 기술/연구
   ▹ 영국 옥스퍼드⼤ 연구 결과, 글로벌 AI 칩 분포의 양극화 현상 심각······························ 12
n 옥스퍼드⼤ 연구진에 따르면 AI 개발에 필수적인 GPU 클러스터는 미국과 중국을 중심으로 
전 세계 30개국에 집중되어 있으며, 대부분 지역에는 GPU 클러스터가 부재
n 영국 옥스퍼드⼤ 연구진이 2024년 8월 22일 공개한 연구 결과에 따르면 AI 개발에 필수적인 
GPU와 같은 AI 칩은 전 세계 30개국에 고도로 집중
∙2024년 8월 8일~ 8월 10일 1,098명의 미국 성인을 대상으로 실시된 온라인 설문조사 결과, 미국 
근로자의 41%는 일자리 시장이 나쁘다고 인식


## 5. 검색기(Retriever) 생성

In [25]:
# 벡터스토어에 있는 정보를 검색하고 생성
retriver = vectorstore.as_retriever()

In [8]:
retriver.invoke("옥스퍼드 대학교에서 어떤 연구를 진행했어?")

[Document(metadata={'source': 'data/SPRi AI Brief_10월호_산업동향_F.pdf', 'file_path': 'data/SPRi AI Brief_10월호_산업동향_F.pdf', 'page': 8, 'total_pages': 25, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13947', 'producer': 'Hancom PDF 1.3.0.547', 'creationDate': "D:20241007090546+09'00'", 'modDate': "D:20241007090546+09'00'", 'trapped': ''}, page_content='추론 역량을 개선해 물리학, 화학, 생물학의 복잡한 벤치마크 과제에서 박사과정 학생과 비슷한 \n성과를 나타냈으며, 수학과 코딩에서도 탁월한 성과를 기록'),
 Document(metadata={'source': 'data/SPRi AI Brief_10월호_산업동향_F.pdf', 'file_path': 'data/SPRi AI Brief_10월호_산업동향_F.pdf', 'page': 1, 'total_pages': 25, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13947', 'producer': 'Hancom PDF 1.3.0.547', 'creationDate': "D:20241007090546+09'00'", 'modDate': "D:20241007090546+09'00'", 'trapped': ''}, page_content='3. 기술/연구\n   ▹ 영국 옥스퍼드⼤ 연구 결과, 글로벌 AI 칩 분포의 양극화 현상 심각······························ 1

## 6. 프롬프트 생성

- 당신은 질문 답변 직업을 위한 어시스턴트입니다.
- 주어진 문맥을 사용하여 질문에 답변하세요.
- 답에 대한 근거를 어디서 찾아왔는지 #Source란에 적어줘.
- 가급적이면 이해하기 쉽게 요약해서 알려줘.
- 답을 모르는 경우에는 모른다고 말씀하세요.
- 한국어로 답변하세요.

In [26]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question.
note the source of the answer(where you got the information from) on #Source
if possible, inform me in summary that I can easily understand.
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:

#Source: 
"""
)

## 7. LLM 모델 생성

In [10]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0)

## 8. Chain 생성

In [27]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain = (
    {"context" : retriver, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
# 체인 실행

question = "주어진 자료의 내용을 주제별로 요약해서 알려줘."
response = chain.invoke(question)

print(response)

주어진 자료의 내용을 주제별로 요약하면 다음과 같습니다:

1. **데이터 주석 작업**: 대규모 데이터셋에 주석을 다는 작업은 시간과 비용이 많이 소요됩니다. (#Source: Page 15)

2. **공공 데이터 활용**: 창의적 수업을 지원하는 프로젝트가 공공 데이터 활용 방식을 변화시키는 첫 번째 사례가 될 것으로 기대됩니다. (#Source: Page 19)

3. **질의응답 작업**: 역사, 과학, 문화 등 다양한 주제에 걸친 대규모 데이터셋을 이용한 질의응답 작업이 이루어지고 있습니다. (#Source: Page 16)

4. **교육 분야 기술 활용**: 콘텐츠 스토어는 교사의 과제 채점, 교육 자료 제작, 학교의 행정 업무 지원 등 교육 분야에 특화된 기술 기업을 대상으로 활용됩니다. (#Source: Page 19)


In [29]:
# 체인 실행

question = "제공된 문서에 대해 자세하게 알려줘"
response = chain.invoke(question)

print(response)

제공된 문서는 2024년 10월호의 "SPRi AI Brief"라는 제목의 산업 동향 보고서입니다. 이 보고서는 주로 인공지능 관련 정책, 기업 및 산업 동향, 기술 및 연구, 인력 및 교육에 관한 내용을 다루고 있습니다. 특히, 하이퍼라이트라는 회사가 오류를 자체 수정하는 '리플렉션 70B'라는 오픈소스 모델을 공개한 내용이 포함되어 있습니다. 이 모델은 추론 과정에서 발생한 오류를 스스로 감지하고 수정하여 정확도를 향상시키는 특징을 가지고 있으며, 여러 벤치마크에서 뛰어난 성능을 보였다고 합니다. 그러나 오픈소스 AI 커뮤니티와 연구자들은 이 모델의 실제 성능에 대해 의혹을 제기하고 있습니다.

#Source: data/SPRi AI Brief_10월호_산업동향_F.pdf, page 13


In [30]:
# 체인 실행

question = "제공된 문서에 대해 자세하게 알려줘"
response = chain.invoke(question)

print(response)

제공된 문서는 2024년 10월호의 "SPRi AI Brief"라는 제목의 산업 동향 보고서입니다. 이 보고서는 다양한 주제를 다루고 있으며, 주요 내용은 다음과 같습니다:

1. **정책/법제**: AI 관련 정책 및 법제 동향을 다루고 있습니다.
2. **기업/산업**: AI 산업 내 기업들의 동향과 관련된 내용을 포함하고 있습니다.
3. **기술/연구**: AI 기술 발전과 연구 결과에 대한 내용을 다루고 있습니다. 예를 들어, 하이퍼라이트라는 회사가 오류를 자체 수정하는 '리플렉션 70B'라는 오픈소스 모델을 공개했다는 내용이 포함되어 있습니다. 이 모델은 추론 과정에서 발생한 오류를 스스로 감지하고 수정하여 정확도를 향상시키는 특징이 있습니다.
4. **인력/교육**: AI와 관련된 인력 및 교육 동향을 다루고 있습니다.

이 외에도 보고서에는 AI 칩 분포의 양극화 현상, LLM의 품질 평가, AI의 일자리 영향 등에 대한 다양한 연구 결과와 전망이 포함되어 있습니다.

#Source: data/SPRi AI Brief_10월호_산업동향_F.pdf, 페이지 0, 1, 13


In [31]:
# 체인 실행

question = "AI의 일자리 영향과 관련하여 자세하게 알려줘"
response = chain.invoke(question)

print(response)

미국 근로자들은 AI 기술 발전이 일자리에 미치는 영향에 대해 다양한 의견을 가지고 있습니다. 유고브의 조사에 따르면, 미국 근로자의 48%는 AI의 발전으로 인해 일자리가 줄어들 것으로 예상하고 있으며, 34%는 AI로 인한 실직이나 근무 시간, 급여 감소를 우려하고 있습니다. 반면, 63%의 근로자는 이러한 우려를 하지 않고 있으며, 실제로 AI로 인한 실직이나 근무 시간, 급여 감소 사례는 극소수에 불과합니다. 또한, AI 발전으로 일자리가 늘어날 것이라고 응답한 비율은 11%, 영향이 없을 것이라는 응답은 28%로 나타났습니다. 

한편, IBM 기업가치연구소는 생성 AI가 직원의 잠재력을 발견하고 업무 흐름을 혁신하여 기업의 새로운 미래를 개척할 수 있다고 보고 있으며, 이를 위해 미래 업무에 대한 비전 수립이 필요하다고 강조하고 있습니다. 

#Source: data/SPRi AI Brief_10월호_산업동향_F.pdf, 페이지 18, 21
